In [10]:
# Дескрипторы данных.

In [11]:
# Давайте посмотрим на то как вызывается метод __get__():
# Вставим принты чтобы посмотреть что передается в эти аргументы:
from time import time

class Epoch:
    def __get__(self, instance, owner_class):
        print(f'Self: {self}')
        print(f'Intance: {instance}')
        print(f'Owner class: {owner_class}')
        return int(time())
    
    
class MyTime:
    epoch = Epoch()

    
m = MyTime()

In [12]:
m.epoch

Self: <__main__.Epoch object at 0x7f4e28a16ca0>
Intance: <__main__.MyTime object at 0x7f4e28a16c40>
Owner class: <class '__main__.MyTime'>


1601922422

In [13]:
# В выводе мы видим, что метод __get__ получил объект класса Epoch, instance получил объект класса MyTime, и третьим аргументом был передан класс собственник, т.к. сам класс MyTime.

In [14]:
# Теперь мы можем обращаться к атрибуту класса через сам класс:
MyTime.epoch

Self: <__main__.Epoch object at 0x7f4e28a16ca0>
Intance: None
Owner class: <class '__main__.MyTime'>


1601922426

In [15]:
# Все тоже самое, кроме параметра инстанс, который не получил ничего, потомучто обращение произошло через класс, а не через экземпляр.
# Такое поведение позволяет нам возвращать разные значения, взависимости от того, откуда было вызвано свойство epoch.
# Если во второй параметр (на место instance) было что-то передано - это один вариант.
# Если ничего не передано - другой вариант.
# Например мы можем возвращать значение свойств, если обращение произошло из экземпляра.
# И возвращать сам экземпляр дескриптора, если произошло обращение через класс.
# Это как раз то поведение, которое реализует property.

In [16]:
# Пример обращения к классу:
class Person:
    _name = 'Oleg'
    @property
    def name(self):
        return self._name

Person.name

In [17]:
# Получили экземпляр класса property.
# А если обратиться к экземпляру, то получим значение:
Person().name

'Oleg'

In [18]:
# На основе такого поведения давайте изменим наш исходный пример:
from time import time

class Epoch:
    def __get__(self, instance, owner_class):
        if instance is None: # здесь важно проверить был ли передан хоть какой-то аргумент или не был передан, т.е. None
            return self
        return int(time())
    
    
class MyTime:
    epoch = Epoch()

    
m = MyTime()

In [19]:
# До этого момента мы по сути говорили о None-data дескрипторах.
# Давайте реализуем метод __set__().
# Для начала надо понять где мы будем хранить данные. Ведь у нас есть 3 варианта.
# Рассмотрим метод __set__, в котором нет параметра owner, т.к. swtter всегда вызывается из экземпляра, но есть третий аргумент - это значение, которое нам надо сохранить:
from time import time

class Epoch:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return int(time())
    
    def __set__(self, instance, value):
        pass
    
    
class MyTime:
    epoch = Epoch()

    
m = MyTime()

In [20]:
# Т.е. у нас остается два варианта. Хранить данные в классе или экземпляре класса.
# Конечно мы выбираем экземпляр.
# Т.к. в случае хранения данных в свойстве класса, все экземпляры будут делить это свойство между собой.
# И когда потребуется его изменить, изменено оно будет для всех экземпляров, т.е. станет для них глобальным объектом.

In [21]:
# Давайте продеманстрируем это:

from time import time

class Epoch:
    def __get__(self, instance, owner_class):
        print(f'id of self: {id(self)}')
        if instance is None:
            return self
        return int(time())
    
    def __set__(self, instance, value):
        pass
    
    
class MyTime:
    epoch = Epoch()

    
m = MyTime()

In [22]:
m2 = MyTime()

In [23]:
m.epoch

id of self: 139973665436576


1601922445

In [24]:
m2.epoch

id of self: 139973665436576


1601922447

In [25]:
# Т.е. два экземпляра класса работают с одним и тем же объектом.
# Так не должно работать.
# Для хранения данных нужны именно экземпляры классов. И по этой же причине методы __get__ и __set__ принимают экземпляры классов.

In [26]:
# Теперь давайте посмотрим, почему нельзя хранить данные в экземпляре дескриптора:
class InDescriptor:
    def __set__(self, instance, value):
        print(f'I got {value}') # пока заглушка
        
    def __get__(self, instance, owner):
        if instance is None:
            print('Call from a class') # пока заглушка
        print('Call from instance') # пока заглушка
        

class Vector:
    x = InDescriptor()
    y = InDescriptor()
    
    
v = Vector()

In [27]:
# Присвоем координате x значение 5:
v.x = 5

I got 5


In [28]:
# Убедились,что метод __set__ вызывается и получает наше значение.
# Теперь нам надо его сохранить.
# Сохранять будет в том месте, которое значение получает, т.е. в экземпляре дескриптора.
class InDescriptor:
    def __set__(self, instance, value):
        self._value = value
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self._value
        

class Vector:
    x = InDescriptor()
    y = InDescriptor()
    
    
v = Vector()

In [29]:
v.x = 5

In [30]:
v2 = Vector()

In [31]:
v2.x

5

In [32]:
# Как видно значение из первого экземпляра доступно и для другого экземпляра.
# Если поменять значение у второго экземпляра, то оно также будет доступно и для первого:
v2.x = 200
v.x

200

In [33]:
# Конечно так не должно работать. Т.к. объекты класса хранят общее состояние в дескрипторе.

In [34]:
# Следовательно мы не можем хранить данные свойств в дескрипторе.

In [41]:
# Также возникает вопрос - как нужно сохранить данные, под каким именем они должны храниться, чтобы это имя можно было определеить например в экземпляре класса, чтобы потом его можно было легко прочитать.
# Причем это надо делать так, чтобы не происходила перезапись какого-либо атрибута или свойства.
# Т.е. надо каким-то образом хранить разные имена для хранения данных разных экземпляров этих дескрипторов.
# Одно из возможных решений - это использовать словарь, в котором в качестве ключа будет экземпляр класса, из которого было обращение к свойству.
# Давайте еще один пример посмотрим, почему это деструктивно:

class IntDescriptor:
    def __init__(self):
        self._values = {} # для каждого экземпляра будет создан пустой словарь
    
    def __set__(self, instance, value):
        self._values[instance] = value # тут обращаемся к словарю и в качестве ключ передаем instance
                                       # напомним, что такой объект должен быть хэшируемым, т.е. для должны быть определены методы __hash__ и __eq__

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self._values.get(instance)
        

class Vector:
    x = IntDescriptor()
    y = IntDescriptor()

In [42]:
v1 = Vector()
v2 = Vector()

In [43]:
v1.x = 5

In [44]:
v2.x

In [45]:
v2.x = 10

In [46]:
v1.x

5

In [47]:
# Все работает как надо.

In [48]:
# Разберем что здесь происходит.

In [50]:
# Свойства x и y являются экземплярами класса IntDescriptor.
# Поэтом каждый из этих экземпляров (и x, и y) получил при инициализации пустой словарь.
# Эти словари не зависят друг от друга.
# Когда мы присваиваем значение экземпляру класса Vector, оно передается в метод __set__, который записывает это значение в словарь.
# В словаре values ключом является экземпляр класса Vector, из которого произошло обращение к свойству.
# Т.е. для хранения здесь используется и self (экземпляр класса дескриптора) и экземпляр класса, из которого произошел вызов свойства.
# Далее вызов дескриптора и соответсвенно вызов метода __get__ или __set__.
# При обращении в свойству через класс мы можем получить доступ к словарю values и посмотреть как оно все работает.

In [52]:
Vector.x._values

{<__main__.Vector at 0x7f4e289b38b0>: 5,
 <__main__.Vector at 0x7f4e289b3bb0>: 10}

In [53]:
# Видим наши объекты со значениями.
# Такая реализация работает неплохо и ожидаемо.
# Но есть существенная проблема - это количество ссылок которое мы создаем.
# Посчитаем количество ссылок произвольных объектов:
import sys

v = 1
sys.getrefcount(v)

3032

In [54]:
val = 'oleg'
sys.getrefcount(val)

3

In [55]:
# Либо можно использовать модуль ctypes:
import ctypes

def ref_count(obj_id):
    return ctypes.c_long.from_address(obj_id).value

ref_count(id(val))

1

In [56]:
# Теперь попробуем посчитать ссылки:
v3 = Vector()
ref_count(id(v3))

1

In [57]:
# Сохраняем id нашего вектора в переменную:
id_v = id(v3)

In [58]:
# Далее ставим эксперимент. Присваивая свойствам x и y какие-то значения мы таким образом будем создавать еще по одной ссылке для каждого экземпляра дескриптора.
# Т.к. в словаре будет храниться ссылка на объект.

In [59]:
v3.x = 5
ref_count(id_v)

2

In [60]:
v3.y = 5
ref_count(id_v)

3

In [61]:
# Удалим экземпляр ветора:
del v3
ref_count(id_v)

2

In [63]:
# Несмотря на удаление объекта, ссылок все равно осталось 2.
# Т.е. гарбейдж каллектор не сможет очистить такую память.

In [65]:
# Но мы можем до него дотянутся:
Vector.x._values

{<__main__.Vector at 0x7f4e289b38b0>: 5,
 <__main__.Vector at 0x7f4e289b3bb0>: 10,
 <__main__.Vector at 0x7f4e289b3ac0>: 5}

In [66]:
# Приведем это все к списку ключей:
list(Vector.x._values.keys())

In [68]:
list(Vector.x._values.keys())[0]

In [69]:
# Это удаленный объект.
# А самое явление ни что иное как утечка памяти!
# Сорщик мусора удаляет объекты из памяти, только если на них не осталось ссылок.
# Чтобы это предотвратить, надо использовать слабые ссылки (weakref).
# Либо использовать вариант дескриптора, который будет разобран через занятие.